<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Hero-Stats-Transformer" data-toc-modified-id="Hero-Stats-Transformer-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Hero Stats Transformer</a></span></li><li><span><a href="#Partidas-Dota-2---Transfomador" data-toc-modified-id="Partidas-Dota-2---Transfomador-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Partidas Dota 2 - Transfomador</a></span></li><li><span><a href="#Join-entre-dataframes" data-toc-modified-id="Join-entre-dataframes-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Join entre dataframes</a></span><ul class="toc-item"><li><span><a href="#Filtrando-game_mode-e-lobby_type" data-toc-modified-id="Filtrando-game_mode-e-lobby_type-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Filtrando game_mode e lobby_type</a></span></li><li><span><a href="#Unificando-datastes" data-toc-modified-id="Unificando-datastes-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Unificando datastes</a></span></li></ul></li><li><span><a href="#Salvando-dataset" data-toc-modified-id="Salvando-dataset-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Salvando dataset</a></span></li></ul></div>

---
# Transformador 2: Anexar os status dos 10 heróis para cada um dos dois times

**Objetivo do experimento:** Anexar os status de cada time ao dataset de treinamento. As seguintes soma features de features por herói serão adicionadas:

- Is_Melle (quantidade de heróis de curto e longo alcance)
- Roles (Número total de roles por time)
- Quantidade de heróis por atributo principal: str, int, agi


Abaixo a *Ordem de procedimentos:*

**Arquivo Hero_stats**
1. Carregar o arquivo hero_stats
2. Filtrar as colunas desejadas
3. Aplicar a transformação nos dados de hero_stats

**Arquivos de partidas**
1. Procurar por todos os dados disponíveis na pasta 'raw_data'
2. Concatenar todos em um único dataframe
3. Remover valores nulos
4. Remover colunas duplicadas
5. Adicionar uma coluna para um herói de cada time

**União de dados**
1. Filtrar apenas por partidas rankeadas e game_mode = 'game_mode_all_draft'
2. Fazer um join entre os datasets
3. Tratar para somar as colunas desejadas de cada time

**Loading libraries and ``hero_stats`` JSON**

In [1]:
data_path = '../data/raw_data'

import pandas as pd
import numpy as np
from datetime import datetime
import os

import sys
sys.path.append('../utils')
import transformer_utils

## Hero Stats Transformer

In [2]:
hero_status_columns =['hero_id', 'is_Melle', 'primary_attr_agi', 'primary_attr_int',
       'primary_attr_str', 'Nuker', 'Disabler', 'Initiator', 'Durable',
       'Support', 'Jungler', 'Carry', 'Pusher', 'Escape']

In [3]:
hero_stats_raw = pd.read_json(data_path+'/hero_stats.json')
hero_stats_df = transformer_utils.hero_stats_tranformer(hero_stats_raw)
hero_stats_df = hero_stats_df.loc[:, hero_status_columns]
hero_stats_df.head()

,hero_id,is_Melle,primary_attr_agi,primary_attr_int,primary_attr_str,Nuker,Disabler,Initiator,Durable,Support,Jungler,Carry,Pusher,Escape
0,1,1,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,2,1,0,0,1,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0
2,3,0,0,1,0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
3,4,1,1,0,0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
4,5,0,0,1,0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


---
## Partidas Dota 2 - Transfomador 

**Searching for each .csv file in the 'raw_data' folder**

In [4]:
lst_df = []
for root, dirs, files in os.walk(data_path):
    for filename in files:
        xlsx_file, file_extension = os.path.splitext(filename)
        print('.csv file found:\n')
        if(file_extension == '.csv'):
            print(filename)
            file_path = root +'\\' + filename
            lst_df.append(pd.read_csv(file_path))  

.csv file found:

21-04-24 16h14m31s.csv
.csv file found:

.csv file found:

.csv file found:



**Drop NA**

In [5]:
match_df = pd.concat(lst_df)

print('Dataframe shape:', match_df.shape)
print('Total nan: \n\n', match_df.isna().sum())

match_df.dropna(inplace=True)
print('\nDataframe shape:', match_df.shape)

Dataframe shape: (46000, 9)
Total nan: 

 Unnamed: 0      0
match_id        0
radiant_win     0
avg_mmr         0
duration        0
lobby_type      0
game_mode       0
radiant_team    0
dire_team       0
dtype: int64

Dataframe shape: (46000, 9)


**Remove duplicated rows**

In [6]:
match_df.drop_duplicates(subset=['match_id'])
print('\nDataframe shape:', match_df.shape)


Dataframe shape: (46000, 9)


**Adding column for each hero on radiant and dire team**

In [7]:
radiant_columns = ['Radiant 1','Radiant 2', 'Radiant 3', 'Radiant 4', 'Radiant 5']
dire_columns = ['Dire 1','Dire 2', 'Dire 3', 'Dire 4', 'Dire 5']

match_df[radiant_columns] = match_df.radiant_team.str.split(",",expand=True,)
match_df[dire_columns] = match_df.dire_team.str.split(",",expand=True,)
print('\nDataframe shape:', match_df.shape)


Dataframe shape: (46000, 19)


**Removing Unnamed and dire_team and radiant_team column**

In [8]:
match_df.drop(columns=['Unnamed: 0', 'dire_team', 'radiant_team', 'match_id'], inplace=True)
print('\nDataframe shape:', match_df.shape)


Dataframe shape: (46000, 15)


**Converting string to numerical**

In [9]:
match_df['radiant_win'] = match_df['radiant_win'].astype(int)
match_df = match_df.apply(pd.to_numeric)
match_df.head()

,radiant_win,avg_mmr,duration,lobby_type,game_mode,Radiant 1,Radiant 2,Radiant 3,Radiant 4,Radiant 5,Dire 1,Dire 2,Dire 3,Dire 4,Dire 5
0,1,3439,1649,7,22,121,119,2,13,70,129,83,80,63,30
1,1,3774,1848,0,22,109,10,108,128,84,123,69,67,73,71
2,1,3311,1951,7,22,129,81,35,88,5,32,135,37,121,126
3,1,3408,995,0,22,83,56,17,62,9,26,32,51,6,135
4,0,4621,1818,7,22,11,100,26,44,2,135,42,25,30,74


## Join entre dataframes

### Filtrando game_mode e lobby_type

In [10]:
game_mode_df = pd.read_json(data_path+'/game_mode.json').T
lobby_type_df = pd.read_json(data_path+'/lobby_type.json').T

dict_game_mode = dict(zip(game_mode_df['id'], game_mode_df['name']))
dict_lobby_type = dict(zip(lobby_type_df['id'], lobby_type_df['name']))

df = match_df

df['game_mode'].replace(dict_game_mode, inplace=True)
df['lobby_type'].replace(dict_lobby_type, inplace=True)


print('Formato do dataframe:', df.shape)
df = df[df['game_mode'] == 'game_mode_all_draft']
df = df[df['lobby_type'] == 'lobby_type_ranked']


df.drop(columns=['game_mode','lobby_type'], inplace=True)
print('Formato do dataframe após remoção:', df.shape)
match_df = df
match_df.reset_index(inplace=True, drop=True)

Formato do dataframe: (46000, 15)
Formato do dataframe após remoção: (32293, 13)


In [11]:
print(match_df.shape)
print(hero_stats_df.shape)

(32293, 13)
(121, 14)


### Unificando datastes

In [12]:
def join_and_summarize(left_df, left_key_columns, right_df, right_key, columns_to_summarize):
    """Apply a join on two dataframes and summarize it's columns """

    shape_summarized = [left_df.shape[0], len(columns_to_summarize)]
    df_summarized = pd.DataFrame(np.zeros(shape_summarized), columns = columns_to_summarize)
    
    for i, left_key in enumerate(left_key_columns):
        join_df = pd.merge(left_df, right_df, left_on=left_key, right_on=right_key)
        df_summarized += join_df.loc[:,columns_to_summarize]
        
    return df_summarized

In [13]:
def create_features_by_team(match_df, hero_stats_df, columns_to_summarize):
    """Concatenade data from dota 2 matches and hero stats to create a dataframe summarized by team"""
    
    radiant_key_columns = ['Radiant 1','Radiant 2', 'Radiant 3', 'Radiant 4', 'Radiant 5']
    dire_key_columns = ['Dire 1','Dire 2', 'Dire 3', 'Dire 4', 'Dire 5']
    right_key = ['hero_id']
    
    # Radiant - Join and Summarize
    radiant_df_summarized = join_and_summarize(match_df, radiant_key_columns, hero_stats_df, right_key, columns_to_summarize)
    radiant_df_summarized = radiant_df_summarized.add_prefix('Radiant_')

    # Dire - Join and Summarize
    dire_df_summarized = join_and_summarize(match_df, dire_key_columns, hero_stats_df, right_key, columns_to_summarize)
    dire_df_summarized = dire_df_summarized.add_prefix('Dire_')
    
    # Concatenate dataframes and remove key columns
    match_df.drop(columns=(dire_key_columns + radiant_key_columns), inplace=True)
    result_df = pd.concat([match_df, radiant_df_summarized, dire_df_summarized], axis=1)
    
    return result_df

In [14]:
columns_to_summarize = columns_to_summarize = ['is_Melle', 'primary_attr_agi', 'primary_attr_int',
                                               'primary_attr_str', 'Nuker', 'Disabler', 'Initiator',
                                                'Durable', 'Support', 'Jungler', 'Carry', 'Pusher', 'Escape']
result_df = create_features_by_team(match_df, hero_stats_df, columns_to_summarize)

In [15]:
result_df.head()

,radiant_win,avg_mmr,duration,Radiant_is_Melle,Radiant_primary_attr_agi,Radiant_primary_attr_int,Radiant_primary_attr_str,Radiant_Nuker,Radiant_Disabler,Radiant_Initiator,...,Dire_primary_attr_str,Dire_Nuker,Dire_Disabler,Dire_Initiator,Dire_Durable,Dire_Support,Dire_Jungler,Dire_Carry,Dire_Pusher,Dire_Escape
0,1,3439,1649,2.0,1.0,3.0,1.0,3.0,5.0,2.0,...,2.0,1.0,3.0,2.0,3.0,2.0,1.0,3.0,1.0,2.0
1,1,3311,1951,2.0,1.0,3.0,1.0,3.0,5.0,2.0,...,2.0,1.0,3.0,2.0,3.0,2.0,1.0,3.0,1.0,2.0
2,0,4621,1818,2.0,1.0,3.0,1.0,3.0,5.0,2.0,...,2.0,1.0,3.0,2.0,3.0,2.0,1.0,3.0,1.0,2.0
3,0,3750,1808,2.0,1.0,3.0,1.0,3.0,5.0,2.0,...,2.0,1.0,3.0,2.0,3.0,2.0,1.0,3.0,1.0,2.0
4,0,3871,1770,2.0,1.0,3.0,1.0,3.0,5.0,2.0,...,2.0,1.0,3.0,2.0,3.0,2.0,1.0,3.0,1.0,2.0


## Salvando dataset
**Saving data frame on 'working data' folder**

In [16]:
working_data_path = '../data/working_data/2_TRA_'
start_file = datetime.now().strftime("%Y-%m-%d")
output_file = working_data_path + start_file + '_working_data.csv'

result_df.to_csv(output_file, index=False)